# Rb87 Polarizability Calcs

## Dynamic polarizability
$\alpha_0$: scalar shift. no dependence on light polarization <br/>
$\alpha_1$: vector shift. zero for linear light or $j_a=0$ <br/>
$\alpha_2$: tensor shift. non-zero for $j_a\geq1$ <br/>

In [1]:
#### libraries
from numpy import *
import matplotlib.pyplot as plt
from arc import *
from sympy.physics.wigner import wigner_6j,wigner_3j,clebsch_gordan

#### local files
from physconsts import *
from rbconsts import *
from amophys import *

In [46]:
# a dictionary of Rb87 hf levels through 5p3/2. "nu" in [GHz], w.r.t. 5s1/2 COM
# hf_levels = [
#     {'n': 5, 'L': 0, 'J': 0.5, 'F': 1, 'nu': -4.271676631815196},
#     {'n': 5, 'L': 0, 'J': 0.5, 'F': 2, 'nu': 2.563005979089114},
#     {'n': 5, 'L': 1, 'J': 0.5, 'F': 1, 'nu': 377106.953053064},
#     {'n': 5, 'L': 1, 'J': 0.5, 'F': 2, 'nu': 377107.769709364},
#     {'n': 5, 'L': 1, 'J': 1.5, 'F': 0, 'nu': 384230.1823946245},
#     {'n': 5, 'L': 1, 'J': 1.5, 'F': 1, 'nu': 384230.2546166565},
#     {'n': 5, 'L': 1, 'J': 1.5, 'F': 2, 'nu': 384230.4115571805},
#     {'n': 5, 'L': 1, 'J': 1.5, 'F': 3, 'nu': 384230.6782093585}
# ]

# restructure this:

# the keys are nested in order n, L, J, F, nu
hf_levels = {5:
                {0: 
                    {0.5:
                        {1: -4.271676631815196,
                         2: 2.563005979089114}
                    },
                 1:
                    {0.5:
                        {1: 377106.953053064,
                         2: 377107.769709364},
                     1.5:
                        {0: 384230.1823946245,
                         1: 384230.2546166565,
                         2: 384230.4115571805,
                         3: 384230.6782093585}
                    }
                }
            }

In [49]:
n=5
l=0
j=0.5
f=1
hf_levels[n][l][j][f]

-4.271676631815196

## Fine structure energy corrections

$A = \frac{g_I}{h n^3 J(J+1)(2L+1)}E_H \alpha^2 \frac{m}{m_P}$

$\Delta E = \frac{h A}{2}(F(F+1) - I(I+1) - J(J+1))$ is the splitting from the c.o.m. energy of the level F. 

In [24]:
gnuc = 1.6778809412165474e-06 # solved for here so not exact S.I. units i guess

Aconst = lambda n,l,j: gnuc*EH*alpha**2*me/(2*pi*hbar*n**3*j*(j+1)*(2*l+1)*mp)
hfshift = lambda n,l,j,f: Aconst(n,l,j)*(f*(f+1)-I*(I+1)-j*(j+1))/2

In [32]:
## 5S1/2 shifts
hfshift(5,0,0.5,1),hfshift(5,0,0.5,2)

(-4.271250000000001, 2.5627500000000003)

In [34]:
## 5P3/2 shifts - these are not correct :(
[hfshift(5,1,1.5,f) for f in [0,1,2,3]]

[-0.8542500000000001,
 -0.6264500000000001,
 -0.17085000000000003,
 0.5125500000000001]

In [35]:
(384230.6782093585**2 - 384232.6782093585**2)/(384230.4844685**2 - 384232.4844685**2)

1.0000005042493443

In [26]:
(EH*alpha**2*me/(3.417*2*pi*hbar*5**3*.5*(.5+1)*(2*0+1)*mp))**-1

1.6778809412165474e-06

In [8]:
Aconst(5,1/2,0)

3.4170000000000007

## Dynamic polarizabilities for hyperfine states
Below, the labels $F,F'$ refer to hyperfine levels $|n,L,J,F\rangle,|n',L',J',F'\rangle$, where the quantum numbers $I,S$ are the same between the two states and hence have been suppressed. Therefore the sums over F' are sums over such levels, rather than over the quantum number F. 

The total hyperfine polarizability, using scalar (S), vector (V), and tensor (T) polarizabilities which are decoupled from the projection $m_F$, is given by:

$\alpha_{F,m_F}(\omega) = \alpha_F^{(S)}(\omega) + \hat{k}\cdot\hat{B}A\frac{A m_F}{2F}\alpha_F^{(V)}(\omega) + \left(3|\hat{\eta}\cdot \hat{B}|^2-1\right)\frac{3m_F^2-F(F+1)}{2F(2F-1)}\alpha_F^{(T)}(\omega)$

where $\hat{B}$ is the quantization axis unit vector, $\hat{\eta}$ is the laser polarization unit vector, $A=0,\pm1$ corresponds to linear and right/left-handed light polarization, and the $m_F$-independent polarizabilities are

$\alpha_F^{(S)} = \sum_{F'}\frac{2\omega_{F,F'}|\langle F || \hat{d} || F'\rangle|^2}{3\hbar(\omega_{F',F}^2-\omega^2)}$

$\alpha_F^{(V)} = \sum_{F'}(-1)^{F+F'+1}\sqrt{\frac{6F(2F+1)}{F+1}}S_{F,F,F'}^{1,1,1}\frac{\omega_{F,F'}|\langle F || \hat{d} || F'\rangle|^2}{\hbar(\omega_{F',F}^2-\omega^2)}$

$\alpha_F^{(T)} = \sum_{F'}(-1)^{F+F'}\sqrt{\frac{40F(2F+1)(2F-1)}{3(F+1)(2F+3)}}S_{F,F,F'}^{1,1,2}\frac{\omega_{F,F'}|\langle F || \hat{d} || F'\rangle|^2}{\hbar(\omega_{F',F}^2-\omega^2)}$

Hyperfine reduced matrix element expressed in the fine-structure basis:

$\langle F || \hat{d} || F'\rangle = (-1)^{1+I+J+F'}\sqrt{(2F'+1)(2F+1)}S_{F1J}^{J'IF'}\langle J || \hat{d} || J'\rangle$

In [ ]:
# the energy corrections for the hyperfine states are small enough
# that the fine structure angular frequencies will be used in the 
# sums

## $\alpha_F^{(V)}(\omega)$

In [ ]:
# our atom
rb = Rubidium87()

## params to go in the function

# the 5s1/2 state
level_a = hf_levels[1] 
n_a = level_a["n"]
j_a = level_a["J"] 
l_a = level_a["L"]
mF_a = 0
q = 1
n_min = 5
n_max = 11 # exclusive max

freqGHz = rb.getTransitionFrequency(5,0,1/2,5,1,3/2) + (-hf_levels[1]["nu"] + hf_levels[6]["nu"] - 2.1e9)

## function body

w_arr = array([
        2*pi*freqGHz
        ]) #2*pi*c/l for l in l_arr]) # ang. freqs

# w_a = 2*pi*level_a["E"]*1e9 # [rad/s], wrt zeropoint at 5s1/2 COM
# if l_a > 0:
#     l_b = [l_a + 1, l_a - 1] # for dipole-allowed transitions with l_a = 0
# else:
#     l_b = [l_a + 1]

j_b_list = [l - .5, l + .5 for l in l_b]
f_b_list = [l - I, l + I for l in j_b]

alpha1 = zeros(pts) 
terms = 0

# program:
# - loop over levels from nmin to nmax
#  - loop over l
#   - loop over j
#    - loop over f

for n_b in range(n_min, n_max):
    for l_b in range(n_b): # runs through n_b - 1, inclusive
        for j_b in [l - .5, l + .5 for l in l_b]: 
            
            # triangle rule and dipole-allowed conditions
            if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1:
                
                for f_b in [l - I, l + I for l in j_b]
                
                try: 
                    w_ba = abs(hf_levels[n_b][l_b][j_b]):
                    # assume all 
                else: # ignore the hf shift
                
                    w_ba = 2*pi*(eVToGHz(rb.getEnergy(n_b, l_b, j_b) \
                                 - rb.getEnergy(n_a, l_a, j_a))*1e9)
                    
                matelem = abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b)*ee*a0)
#                     print(f"< n={n_a}, l={l_a}, j={j_a} | x | n'={n_b}, l'={l_b}, j'={j_b} >")
                alpha0[i] += (2/(3*hbar*(2*j_a+1)))* \
                                w_ba*matelem**2/(w_ba**2 - w_arr[i]**2) # Mark's Notes 9.11a
                terms += 1
                print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-30)} [Ang.^3], {terms} terms in sum")


## Finite sum for $\alpha_0(\omega)$ - J-M basis

In [35]:
# our atom
rb = Rubidium87()

# the 5s1/2 state
level_a = hf_levels[1] 
n_a = level_a["n"]
j_a = level_a["J"] 
l_a = level_a["L"]
# w_a = 2*pi*level_a["E"]*1e9 # [rad/s], wrt zeropoint at 5s1/2 COM

l_b = l_a + 1 # for dipole-allowed transitions with l_a = 0
j_b_list = [ l_b - .5, l_b + .5]
n_min = 5
n_max = 11 # exclusive max

f780A = rb.getTransitionFrequency(5,0,1/2,5,1,3/2) + (-hf_levels[1]["nu"] + hf_levels[6]["nu"] - 2.1e9)

# l_arr = [rb.getTransitionWavelength(
#             n_a,l_a,j_a,n_b,l_b,j_b
#         )] # 1.064e-6] # selected wavelengths
w_arr = array([
        2*pi*f780A
    ]) #2*pi*c/l for l in l_arr]) # ang. freqs
mF_a = 0
q = 1
mF_b = mF_a + q

pts = len(w_arr)
alpha0 = zeros(pts) 
terms = 0

for i in range(pts):
    for n_b in range(n_min, n_max):
        for l_b in range(n_b): # runs through n_b - 1, inclusive
            for j_b in j_b_list: 
                
                # triangle and dipole-allowed conditions
                if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1: 
                    w_ba = 2*pi*(eVToGHz(rb.getEnergy(n_b, l_b, j_b) \
                                        - rb.getEnergy(n_a, l_a, j_a))*1e9)
                    matelem = abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b)*ee*a0)
#                     print(f"< n={n_a}, l={l_a}, j={j_a} | x | n'={n_b}, l'={l_b}, j'={j_b} >")
                    alpha0[i] += (2/(3*hbar*(2*j_a+1)))* \
                                    w_ba*matelem**2/(w_ba**2 - w_arr[i]**2) # Mark's Notes 9.11a
                    terms += 1
                    print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-30)} [Ang.^3], {terms} terms in sum")
# alpha as printed is in cubic Angstroms. as it stands, I don't remember how to make it true S.I.
#A^3 to cgs: 1e-30 -> 1e-6; cgs to a.u.: EH/(ee*a0)**2

alpha0 ~= -402.7888132803234 [Ang.^3], 1 terms in sum
alpha0 ~= 2740938.8660773896 [Ang.^3], 2 terms in sum
alpha0 ~= 2740938.93768191 [Ang.^3], 3 terms in sum
alpha0 ~= 2740939.1255553644 [Ang.^3], 4 terms in sum
alpha0 ~= 2740939.132091927 [Ang.^3], 5 terms in sum
alpha0 ~= 2740939.1522206077 [Ang.^3], 6 terms in sum
alpha0 ~= 2740939.153771693 [Ang.^3], 7 terms in sum
alpha0 ~= 2740939.159256726 [Ang.^3], 8 terms in sum
alpha0 ~= 2740939.1608327376 [Ang.^3], 9 terms in sum
alpha0 ~= 2740939.165618111 [Ang.^3], 10 terms in sum
alpha0 ~= 2740939.1664394536 [Ang.^3], 11 terms in sum
alpha0 ~= 2740939.1689963783 [Ang.^3], 12 terms in sum


In [14]:
# mine = alpha0[0]/(4*pi*e0*1e-6) # cgs
# marks = 97e-24 # for 1064nmb
# print(mine,marks)
# print(mine/marks-1)

9.992826185051465e-23 9.7e-23
0.03018826650015094


## Finite sum for $\alpha_1(\omega)$ - J-M basis

In [ ]:
# our atom
rb = Rubidium87()

# the 5p3/2 state
level_a = hf_levels[6] 
n_a = level_a["n"]
j_a = level_a["J"] 
l_a = level_a["L"]
# w_a = 2*pi*level_a["E"]*1e9 # [rad/s], wrt zeropoint at 5s1/2 COM

l_b = l_a + 1 # for dipole-allowed transitions with l_a = 0
j_b_list = [ l_b - .5, l_b + .5]
n_min = 5
n_max = 11 # exclusive max

f780A = rb.getTransitionFrequency(5,0,1/2,5,1,3/2) + (-hf_levels[1]["nu"] + hf_levels[6]["nu"] - 2.1e9)

# l_arr = [rb.getTransitionWavelength(
#             n_a,l_a,j_a,n_b,l_b,j_b
#         )] # 1.064e-6] # selected wavelengths
w_arr = array([
        2*pi*f780A
    ]) #2*pi*c/l for l in l_arr]) # ang. freqs
mF_a = 0
q = 1
mF_b = mF_a + q

pts = len(w_arr)
alpha1 = zeros(pts) 
terms = 0

for i in range(pts):
    for n_b in range(n_min, n_max):
        for l_b in range(n_b): # runs through n_b - 1, inclusive
            for j_b in j_b_list: 
                
                # triangle and dipole-allowed conditions
                if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1: 
                    w_ba = 2*pi*(eVToGHz(rb.getEnergy(n_b, l_b, j_b) \
                                        - rb.getEnergy(n_a, l_a, j_a))*1e9)
                    matelem = abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b)*ee*a0)
#                     print(f"< n={n_a}, l={l_a}, j={j_a} | x | n'={n_b}, l'={l_b}, j'={j_b} >")
                    alpha0[i] += (2/(3*hbar*(2*j_a+1)))* \
                                    w_ba*matelem**2/(w_ba**2 - w_arr[i]**2) # Mark's Notes 9.11a
                    terms += 1
                    print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-30)} [Ang.^3], {terms} terms in sum")
# alpha as printed is in cubic Angstroms. as it stands, I don't remember how to make it true S.I.
#A^3 to cgs: 1e-30 -> 1e-6; cgs to a.u.: EH/(ee*a0)**2

### AC Stark Shift on $|5 s_{1/2} F=2, m_F=0>$
due to the Rydberg 960 and 780A beams

In [25]:
alpha0_780A = 4*pi*e0*1e-6*1.1653070912238685e-18
alpha0_480 = 4*pi*e0*1e-6*4.794264552830031e-07

efield = lambda intensity: sqrt(2*intensity/(c*e0)) 

def beam_int(z, wx, wy, lmbda, P, xy_dz=0, r=0):
    zRx = pi*wx**2/lmbda
    zRy = pi*wy**2/lmbda
    
    wwx = wx**2*(1+(z/zRx)**2)
    wwy = wy**2*(1+(z/zRy)**2)
    return (P/(pi*wx*wy))*exp(-r**2/wwx)*exp(-r**2/wwy)/sqrt(wwx*wwy)
e780A = efield(beam_int(0, 6e-6, 8e-6, l_780A, 8e-6, xy_dz=200e-6))
e480 = efield(beam_int(0, 5e-6, 5e-6, l_480, 20e-3))

# acstark
acstark0 = -.25*(alpha0_780A*e780A**2 + alpha0_480*e480**2)
acstark0

-0.10234844913314059

## Finite sum for $\alpha_0(\omega)$, 5s1/2 - Hyperfine basis
This doesn't come out correctly.. :/

In [17]:
#TODO: refactor this to make it much more user friendly
# - define a function withs args for state (n, l, j, f), 
# the uppermost state (n', l', j', f'), and wavelengths
# and polarization of the incident light

# our atom
rb = Rubidium(87)

# the 5s1/2,F=2,mF=0 state 
level_a = hf_levels[1] 
n_a = level_a["n"]
j_a = level_a["J"] 
l_a = level_a["L"]
f_a = level_a["F"]
mf_a = 0 # I'm not sure these ever get used.. might be defined below
mf_b = 0



# for the L-S coupling part (states beyond 5p3/2)
l_b = l_a + 1 # for dipole-allowed transitions with l_a = 0
j_b_list = [ l_b - .5, l_b + .5] 

n_min = 5
n_mid = 6
n_max = 11 # exclusive max

l_arr = [1.064e-6] # selected wavelengths, [m]
w_arr = array([2*pi*c/l for l in l_arr]) # ang. freqs
mF_a = 0
q = 0
mF_b = mF_a + q


pts = len(l_arr)
alpha0 = zeros(pts) 
terms = 0
for i in range(len(l_arr)):

    for level_b in hf_levels: # loop over states up through 5p3/2 band (inclusive)
        
        n_b = level_b["n"]
        j_b = level_b["J"] 
        l_b = level_b["L"]
        f_b = level_b["F"]
        
        # triangle and dipole-allowed conditions
        if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1 and abs(f_b - f_a) == q: 
            
            w_ba = 2*pi*(level_b["nu"] - level_a["nu"])*1e9
                        
            # matrix elems squared for hyperfine terms (Mark's notes, after 9.17)
            h0 = (2*f_a + 1)**2*sum([abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b))**2 \
                    *wigner_6j(j_b,f_a,I,f_a,j_a,1)**2 for j_b in j_b_list])*(ee*a0)**2
            
#            print(f"<n={n_a},l={l_a},j={j_a},f={f_a}|x|n'={n_b},l'={l_b},j'={j_b},f'={f_b}>")
            alpha0[i] += (2/(3*hbar*(2*f_a+1)))*w_ba*h0 \
                            /(w_ba**2 - w_arr[i]**2) # Mark's Notes 9.16
            terms += 1
            print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-6)}, {terms} terms in sum")
            
    for n_b in range(n_mid, n_max):
        for l_b in range(n_b): # runs through n_b - 1, inclusive
            for j_b in j_b_list: 

                # triangle and dipole-allowed conditions
                if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1: 
                    w_ba = 2*pi*(eVToGHz(rb.getEnergy(n_b, l_b, j_b) \
                                        - rb.getEnergy(n_a, l_a, j_a))*1e9)
                    matelem = abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b)*ee*a0)
#                     print(f"< n={n_a}, l={l_a}, j={j_a} | x | n'={n_b}, l'={l_b}, j'={j_b} >")
                    alpha0[i] += (2/(3*hbar*(2*j_a+1)))* \
                                    w_ba*matelem**2/(w_ba**2 - w_arr[i]**2) # for q = 0. 
                    
                    terms += 1
#                     print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-6)}, {terms} terms in sum")
print(f"for {level_a}")

alpha0 ~= 3.468244812542488e-23, 1 terms in sum
alpha0 ~= 6.721196582390219e-23, 2 terms in sum
for {'n': 5, 'L': 0, 'J': 0.5, 'F': 2, 'nu': 2.563005979089114}


In [70]:
# 5s1/2,F=2: alpha0 ~= 6.746911307357744e-23, 12 terms in sum (with F=F')
# 5s1/2,F=1: alpha0 ~= 6.736462307586051e-23, 12 terms in sum (with F=F')

In [52]:
3.682966809168338e-23, 16 terms in sum

0.5

In [37]:
(hf_levels[4]["nu"] - hf_levels[0]["nu"])*1e9

384234454071256.3

In [40]:
help(hf_coupling)

<function amophys.hf_coupling(F, mF, J, q, FF, mFF, JJ, I, RME=None)>

In [57]:
h0 = (2*f_a + 1)**2*sum([
        rb.getReducedMatrixElementJ(
            n_a, l_a, j_a, n_b, l_b, j_b
            )*wigner_6j(j_b,f_a,I,f_a,j_a,1)*ee*a0
        for j_b in j_b_list])**2

6j-Symbol is not triangular!
6j-Symbol is not triangular!


In [58]:
del h0